
### CPC‑MS マルチエージェント DEMO（**リファクタ済み・厳密化版**）

**目的**：CPC‑MS（Collective Predictive Coding）と整合する最小限だが**厳密化な実装**。  
**到達点**：
- 内部表象 \(z\) と外的表現 \(w\) を介した **混合モデル**（**Poisson×Gaussian** の**複合尤度**）による **EM 推論**
- **MH ネーミングゲーム**の**意味的提案分布**（語彙中心の**語義方向**へ提案）と **尤度比**受理
- **IPW 重み付け**を伴う **二段推定**による **総効果 / 直接効果 / 媒介効果**の分解
- **Active Inference**：**モンテカルロ近似**での**期待情報利得（EIG）**評価に基づく**次サンプル窓**提案
- **チェックポイント / 安全化シリアライズ**、**オンライン学習（指数フォーゲッティング）**

> **注**：外部 API や巨大モデルが不要な **自給自足**バージョンです。埋め込みは**ハッシュ TF‑IDF**で再現性のある**決定的ベクトル**を実装。必要に応じて外部埋め込みに差し替え可能な**プラグ**構造。

---

### 参考（CPC‑MS 対応）
- 生成：\(p(w,z,o)=p(w)\,p(z\!\mid\!w)\,p(o\!\mid\!z)\)（CPC‑MS 図3）  
- EM：\(o\to z\to w\) の最尤・変分近似（式(2.1)–(2.7)の骨格）  
- MHNG（付録B）：尤度比に基づく受理 \(\min(1, p(Z|w')/p(Z|w))\)  
- 期待自由エネルギー & 情報利得（式(2.8)–(2.10) の要旨）




### 使い方（クイックスタート）

1. **ライブラリ不要**：標準 Python + `numpy`, `pandas`, `matplotlib` のみで動作（追加インストール不要）。  
2. `CONFIG` セルで**窓数・媒体・クラスタ数**などを設定し、**上から順に「Run all」**。  
3. 主要出力：
   - **EM 訓練ログ**（NLL, JS ダイバージェンス, 受理率）
   - **語彙中心 \(w_j\)** と**クラスタ割当**（ソフト/ハード）
   - **因果分解**（総/直接/媒介の推定量と信頼区間・感度）
   - **Active Inference** の**次取得候補**（EIG 上位）
   - **チェックポイント**：`artifacts/` 下に `npz/json` 保存

> 実データに差し替える場合は、**「2. データ取り込み/生成」**の `DataBuilder` を上書きしてください。


In [1]:

# =============================================================
# 0. Imports / Config
# =============================================================
from __future__ import annotations

import math
import json
import os
import random
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True, linewidth=140)
random.seed(42)
np.random.seed(42)

# 全体設定（最小例）
@dataclass
class Config:
    T: int = 200                     # 時間窓数
    sources: Tuple[str, ...] = ("X", "YouTube", "TikTok", "Newspaper", "TV")
    stance_keys: Tuple[str, ...] = ("support", "critic", "neutral")
    emb_dim: int = 256               # ハッシュ TF‑IDF 埋め込み次元
    K: int = 4                       # クラスタ数（語彙サイズ）
    em_max_iter: int = 50
    em_tol: float = 1e-4
    forgetting: float = 0.995        # オンライン更新の指数フォーゲッティング
    mh_alpha: float = 0.35           # 語義方向への提案強度（0..1）
    mh_trials: int = 10              # MH 提案試行回数/ラウンド
    active_mc_samples: int = 64      # Active Inference の MC サンプル数
    holdout_ratio: float = 0.2
    out_dir: str = "artifacts"

CONFIG = Config()

os.makedirs(CONFIG.out_dir, exist_ok=True)
print(CONFIG)


Config(T=200, sources=('X', 'YouTube', 'TikTok', 'Newspaper', 'TV'), stance_keys=('support', 'critic', 'neutral'), emb_dim=256, K=4, em_max_iter=50, em_tol=0.0001, forgetting=0.995, mh_alpha=0.35, mh_trials=10, active_mc_samples=64, holdout_ratio=0.2, out_dir='artifacts')



### 1. データ取り込み / 生成（ダミー時系列 + 要約テキスト）

- 5媒体 × \(T\) 窓の**イベント数（support/critic/neutral）**を **Poisson** で生成。  
- クロス媒体の**ラグ効果**（例：TikTok → X、X → YouTube）を**潜在因子**経由で注入。  
- 各窓テキストを**素朴に合成**（語彙トークン + 業界語）→ **ハッシュ TF‑IDF**で埋め込み。

> 実データ利用時は `DataBuilder.build()` で **(counts_df, texts_df)** を作成してください。


In [2]:

# =============================================================
# 1. DataBuilder: synthetic multi-source time series & texts
# =============================================================
@dataclass
class BuiltData:
    counts: pd.DataFrame   # columns: ['t','source','support','critic','neutral']
    texts: pd.DataFrame    # columns: ['t','source','text']

class DataBuilder:
    def __init__(self, cfg: Config):
        self.cfg = cfg

    def _latent_driver(self, T: int) -> np.ndarray:
        '''潜在因子（トレンド + 周期 + ノイズ）'''
        t = np.arange(T)
        trend = 0.03 * t / T
        seasonal = 0.4 * np.sin(2 * np.pi * t / 24.0)
        noise = 0.15 * np.random.randn(T)
        return np.clip(1.0 + trend + seasonal + noise, 0.2, None)

    def build(self) -> BuiltData:
        T = self.cfg.T
        sources = list(self.cfg.sources)
        S = len(sources)

        driver = self._latent_driver(T)

        # ベース率（媒体×スタンス）
        base = {
            "X":        (18, 12, 10),
            "YouTube":  (12, 10, 14),
            "TikTok":   (20, 12, 11),
            "Newspaper":(8,  10, 16),
            "TV":       (10, 12, 12),
        }

        rows = []
        # クロス媒体ラグ効果：TikTok→X、X→YouTube（係数とラグ）
        lag = 1
        tiktok_series = np.zeros(T)
        x_series = np.zeros(T)

        for t in range(T):
            for s in sources:
                lam_sup, lam_cri, lam_neu = base[s]
                # 潜在因子の影響
                scale = driver[t]
                # クロス媒体ラグ注入
                if s == "X" and t - lag >= 0:
                    scale += 0.2 * (tiktok_series[t - lag] / (lam_sup + lam_cri + lam_neu))
                if s == "YouTube" and t - lag >= 0:
                    scale += 0.15 * (x_series[t - lag] / (lam_sup + lam_cri + lam_neu))

                # Poisson 発生
                sup = np.random.poisson(max(lam_sup * scale, 0.1))
                cri = np.random.poisson(max(lam_cri * scale, 0.1))
                neu = np.random.poisson(max(lam_neu * scale, 0.1))

                rows.append((t, s, sup, cri, neu))

                if s == "TikTok":
                    tiktok_series[t] = sup + cri + neu
                if s == "X":
                    x_series[t] = sup + cri + neu

        counts = pd.DataFrame(rows, columns=["t","source","support","critic","neutral"])

        # テキスト合成（very simple; 実運用は外部埋め込み可）
        def synth_text(row):
            s = row["source"]
            sup, cri, neu = int(row["support"]), int(row["critic"]), int(row["neutral"])
            dominant = max([("support", sup), ("critic", cri), ("neutral", neu)], key=lambda x: x[1])[0]
            tokens = []
            tokens += ["topicA"] * (sup // 3) + ["topicB"] * (cri // 3) + ["topicC"] * (neu // 3)
            tokens += [f"{s}_signal"] * 2
            tokens += [dominant] * 3
            return " ".join(tokens) if tokens else f"{s}_signal neutral"

        texts = counts.copy()[["t","source"]]
        texts["text"] = counts.apply(synth_text, axis=1)
        return BuiltData(counts=counts, texts=texts)

data = DataBuilder(CONFIG).build()
print(data.counts.head())
print(data.texts.head())


   t     source  support  critic  neutral
0  0          X       25      15        9
1  0    YouTube       13      14       17
2  0     TikTok       28      11       13
3  0  Newspaper        5      14       12
4  0         TV       14      14       13
   t     source                                               text
0  0          X  topicA topicA topicA topicA topicA topicA topi...
1  0    YouTube  topicA topicA topicA topicA topicB topicB topi...
2  0     TikTok  topicA topicA topicA topicA topicA topicA topi...
3  0  Newspaper  topicA topicB topicB topicB topicB topicC topi...
4  0         TV  topicA topicA topicA topicA topicB topicB topi...



### 2. 埋め込み（ハッシュ TF‑IDF：決定的・外部依存なし）

- トークンを整数ハッシュに写像（`emb_dim`）  
- TF（正規化）× IDF（コーパスから算出）  
- **再現性**のため、Python の `hash()` ではなく **安定ハッシュ**を実装（FNV‑1a）。


In [3]:

# =============================================================
# 2. Hashed TF‑IDF Embedding (deterministic; no external deps)
# =============================================================
def fnv1a_64(s: str) -> int:
    '''Deterministic 64-bit FNV-1a hash for stable token hashing across sessions.'''
    h = 0xcbf29ce484222325
    fnv_prime = 0x100000001b3
    for c in s.encode('utf-8'):
        h ^= c
        h = (h * fnv_prime) & 0xFFFFFFFFFFFFFFFF
    return h

class HashedTfidf:
    def __init__(self, dim: int = 256):
        self.dim = dim
        self.idf_: Optional[np.ndarray] = None
        self.df_: Optional[np.ndarray] = None
        self.n_docs_: int = 0

    def _tokenize(self, text: str) -> List[str]:
        # 低コストトークナイザ（空白 + 簡易クレンジング）
        return [tok.lower() for tok in text.replace("/", " ").replace("_", " ").split() if tok.strip()]

    def _index(self, token: str) -> int:
        return fnv1a_64(token) % self.dim

    def fit(self, docs: List[str]) -> "HashedTfidf":
        self.n_docs_ = len(docs)
        df = np.zeros(self.dim, dtype=np.float64)
        for text in docs:
            seen = set()
            for tok in self._tokenize(text):
                idx = self._index(tok)
                if idx not in seen:
                    df[idx] += 1.0
                    seen.add(idx)
        self.df_ = df
        # smoothing
        self.idf_ = np.log((self.n_docs_ + 1.0) / (df + 1.0)) + 1.0
        return self

    def transform(self, docs: List[str]) -> np.ndarray:
        assert self.idf_ is not None, "Call fit() first."
        X = np.zeros((len(docs), self.dim), dtype=np.float64)
        for i, text in enumerate(docs):
            counts = np.zeros(self.dim, dtype=np.float64)
            tokens = self._tokenize(text)
            n = len(tokens) if tokens else 1
            for tok in tokens:
                counts[self._index(tok)] += 1.0
            tf = counts / n
            X[i, :] = tf * self.idf_
        # L2 normalize for cosine-like geometry
        norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
        return X / norms

embedder = HashedTfidf(dim=CONFIG.emb_dim).fit(data.texts["text"].tolist())
E = embedder.transform(data.texts["text"].tolist())  # (T*S, dim)
print("Embedding shape:", E.shape)


Embedding shape: (1000, 256)



### 3. 特徴合成 \(x_t = [\text{counts}, \text{embedding}]\)

- **観測 \(o_t\)**：`support, critic, neutral` の**カウント**  
- **内部表象 \(z_t\)**：テキストの **ハッシュ TF‑IDF 埋め込み**  
- **CPC**：\(p(o|z)\) を **Poisson**、\(p(z|w)\) を **Gaussian** とする**複合尤度**で EM。


In [4]:

# =============================================================
# 3. Assemble features & indexing
# =============================================================
# Align rows between counts and texts
df = data.counts.merge(data.texts, on=["t","source"], how="inner")

# Design matrices
count_cols = list(CONFIG.stance_keys)
C = len(count_cols)
counts = df[count_cols].astype(float).to_numpy()  # (N, C)
embeds = E                                     # (N, D)
N, D = embeds.shape

# Useful indexers
sources = df["source"].to_numpy()
times = df["t"].to_numpy()

print(f"N={N}, C={C}, D={D}, sources={set(sources)}")


N=1000, C=3, D=256, sources={'Newspaper', 'YouTube', 'TikTok', 'TV', 'X'}



### 4. CPC 混合モデル（**Poisson×Gaussian** 複合尤度）— EM

- **E-step**：
  \[
  \gamma_{tj} \propto \pi_j\;
  \underbrace{\prod_k \text{Pois}(c_{tk};\lambda_{jk})}_{p(o|z)}
  \underbrace{\mathcal N(e_t; w_j, \sigma_j^2 I)}_{p(z|w)}
  \]
- **M-step**：
  \[
  \pi_j = \frac{1}{N}\sum_t\gamma_{tj},\;
  \lambda_{jk} = \frac{\sum_t \gamma_{tj}\,c_{tk}}{\sum_t \gamma_{tj}},\;
  w_j = \frac{\sum_t \gamma_{tj}\,e_t}{\sum_t \gamma_{tj}},\;
  \sigma_j^2 = \frac{\sum_t \gamma_{tj}\,\|e_t-w_j\|^2}{D\sum_t \gamma_{tj}}
  \]
- **忘却**：時間 \(t\) に対し重み \(\rho^{(T-1-t)}\) を掛け、**概念ドリフト**に対応。
- 指標：**NLL**, **JS ダイバージェンス**（語彙の分離度）。


In [5]:

# =============================================================
# 4. Composite Mixture Model (EM)
# =============================================================
from dataclasses import dataclass

@dataclass
class CPCParams:
    pi: np.ndarray          # (K,)
    lambdas: np.ndarray     # (K, C)  Poisson rates
    mu: np.ndarray          # (K, D)  Gaussian means for embeddings
    sigma2: np.ndarray      # (K,)    isotropic variance

@dataclass
class EMReport:
    nll: float
    js: float
    it: int
    converged: bool

def logsumexp(a: np.ndarray, axis: int = -1) -> np.ndarray:
    m = np.max(a, axis=axis, keepdims=True)
    s = np.log(np.sum(np.exp(a - m), axis=axis, keepdims=True)) + m
    return s if axis is None else np.squeeze(s, axis=axis)

def js_divergence(means: np.ndarray) -> float:
    '''クラスタ中心の分離度を簡易評価（中心間コサインのJS風スコア）。'''
    # cosine similarity matrix
    M = means / (np.linalg.norm(means, axis=1, keepdims=True) + 1e-12)
    S = M @ M.T
    # off-diagonal average similarity -> larger means less separated
    K = means.shape[0]
    off = (np.sum(S) - K) / (K*(K-1) + 1e-9)
    # return (1 - similarity) as separation proxy
    return float(1.0 - off)

class CPCMixture:
    def __init__(self, K: int, C: int, D: int, forgetting: float = 1.0, seed: int = 42):
        self.K, self.C, self.D = K, C, D
        self.forgetting = forgetting
        rng = np.random.RandomState(seed)
        self.params = CPCParams(
            pi=np.ones(K)/K,
            lambdas=rng.gamma(shape=2.0, scale=1.0, size=(K, C)) + 1.0,
            mu=rng.normal(0, 0.5, size=(K, D)),
            sigma2=np.ones(K) * 0.5
        )

    def _log_poisson(self, counts: np.ndarray, lam: np.ndarray) -> np.ndarray:
        # counts: (N, C), lam: (K, C) -> (N, K)
        # log Pois(x; lam) = x log lam - lam - log x!
        # drop constant log x! across K
        xloglam = counts[:, None, :] * np.log(lam[None, :, :] + 1e-12)
        part = np.sum(xloglam - lam[None, :, :], axis=2)
        return part  # (N, K)

    def _log_gauss(self, embeds: np.ndarray, mu: np.ndarray, sigma2: np.ndarray) -> np.ndarray:
        # embeds: (N, D), mu: (K, D), sigma2: (K,) -> (N, K)
        diff = embeds[:, None, :] - mu[None, :, :]
        sq = np.sum(diff*diff, axis=2)
        log_norm = -0.5 * self.D * np.log(2*np.pi) - 0.5 * self.D * np.log(sigma2[None, :] + 1e-12)
        log_exp = -0.5 * (sq / (sigma2[None, :] + 1e-12))
        return log_norm + log_exp  # (N, K)

    def e_step(self, counts: np.ndarray, embeds: np.ndarray, time_index: np.ndarray) -> Tuple[np.ndarray, float]:
        p = self.params
        log_like = (
            self._log_poisson(counts, p.lambdas) +
            self._log_gauss(embeds, p.mu, p.sigma2) +
            np.log(p.pi[None, :] + 1e-12)
        )  # (N, K)

        # Numerically stable responsibilities
        log_norm = logsumexp(log_like, axis=1)  # (N,)
        gamma = np.exp(log_like - log_norm[:, None])  # (N, K)
        nll = -float(np.sum(log_norm))  # negative log-likelihood

        # Apply exponential forgetting weights by time (more recent -> weight 1)
        if self.forgetting < 1.0:
            T_max = int(np.max(time_index))
            ages = (T_max - time_index).astype(np.int64)
            w = np.power(self.forgetting, ages)  # (N,)
            gamma = gamma * w[:, None]
            gamma = gamma / (np.sum(gamma, axis=1, keepdims=True) + 1e-12)
        return gamma, nll

    def m_step(self, counts: np.ndarray, embeds: np.ndarray, gamma: np.ndarray):
        p = self.params
        Nk = np.sum(gamma, axis=0) + 1e-12  # (K,)

        # Update pi
        p.pi = Nk / np.sum(Nk)

        # Update Poisson rates
        p.lambdas = (gamma.T @ counts) / Nk[:, None]
        p.lambdas = np.clip(p.lambdas, 1e-6, None)

        # Update Gaussian means
        p.mu = (gamma.T @ embeds) / Nk[:, None]

        # Update isotropic variance
        diff = embeds[:, None, :] - p.mu[None, :, :]
        sq = np.sum(diff * diff, axis=2)              # (N,K)
        p.sigma2 = np.sum(gamma * sq, axis=0) / (Nk * self.D) + 1e-9

    def fit(self, counts: np.ndarray, embeds: np.ndarray, time_index: np.ndarray, 
            max_iter: int = 100, tol: float = 1e-4) -> EMReport:
        prev_nll = np.inf
        converged = False
        for it in range(1, max_iter+1):
            gamma, nll = self.e_step(counts, embeds, time_index)
            self.m_step(counts, embeds, gamma)
            # Monitor separation
            js = js_divergence(self.params.mu)
            print(f"[EM] it={it:03d} NLL={nll:.3f} JS={js:.3f}")
            if prev_nll - nll < tol:
                converged = True
                print("[EM] Converged.")
                return EMReport(nll=nll, js=js, it=it, converged=converged)
            prev_nll = nll
        return EMReport(nll=nll, js=js, it=it, converged=converged)

    def hard_assign(self, counts: np.ndarray, embeds: np.ndarray, time_index: np.ndarray) -> np.ndarray:
        gamma, _ = self.e_step(counts, embeds, time_index)
        return np.argmax(gamma, axis=1)  # (N,)

    def log_likelihood(self, counts: np.ndarray, embeds: np.ndarray) -> float:
        p = self.params
        log_like = (
            self._log_poisson(counts, p.lambdas) +
            self._log_gauss(embeds, p.mu, p.sigma2) +
            np.log(p.pi[None, :] + 1e-12)
        )
        return float(np.sum(logsumexp(log_like, axis=1)))



### 5. Network（MH ネーミングゲーム）：**意味的提案分布**

- **提案**：クラスタ \(j\) の代表テキストから得た**語義方向** \(s_j\) に、
  \[
  w'_j = (1-\alpha) w_j + \alpha\, s_j
  \]
  を適用（他クラスタの \(w\) は固定）。
- **受理率**：CPC‑MS 付録Bの原理に沿い、全データに対する**尤度比**
  \[
  r = \min\!\left(1, \frac{p(Z|w')}{p(Z|w)}\right)
  \]
  で判断（**提案分布に依存させない**）。


In [6]:

# =============================================================
# 5. MH Naming Game (semantic proposals)
# =============================================================
from dataclasses import dataclass

@dataclass
class MHLog:
    trials: int
    accepted: int
    accept_rate: float
    ll_before: float
    ll_after: float

class MHNamingGame:
    def __init__(self, model: CPCMixture, alpha: float = 0.3):
        self.model = model
        self.alpha = alpha

    def _cluster_semantic_direction(self, embeds: np.ndarray, gamma: np.ndarray) -> np.ndarray:
        '''クラスタごとの '語義方向'（代表的テキスト埋め込み平均）'''
        K, D = self.model.K, self.model.D
        Nk = np.sum(gamma, axis=0) + 1e-12
        S = (gamma.T @ embeds) / Nk[:, None]  # (K,D)
        # L2 normalize
        S = S / (np.linalg.norm(S, axis=1, keepdims=True) + 1e-12)
        return S

    def step(self, counts: np.ndarray, embeds: np.ndarray, time_index: np.ndarray, trials: int = 5) -> MHLog:
        mdl = self.model
        p = mdl.params
        gamma, _ = mdl.e_step(counts, embeds, time_index)
        semantic = self._cluster_semantic_direction(embeds, gamma)

        accepted = 0
        ll_before = mdl.log_likelihood(counts, embeds)

        for _ in range(trials):
            j = np.random.randint(0, mdl.K)
            old_mu_j = p.mu[j].copy()
            # propose along semantic direction
            prop = (1.0 - self.alpha) * old_mu_j + self.alpha * semantic[j]
            p.mu[j] = prop

            ll_after = mdl.log_likelihood(counts, embeds)
            # acceptance ratio
            r = math.exp(ll_after - ll_before)
            if np.random.rand() < min(1.0, r):
                # accept
                ll_before = ll_after
                accepted += 1
            else:
                # reject
                p.mu[j] = old_mu_j

        return MHLog(trials=trials, accepted=accepted, accept_rate=accepted/max(trials,1), ll_before=ll_before, ll_after=ll_before)

# 学習 & MH 提案
mdl = CPCMixture(K=CONFIG.K, C=C, D=D, forgetting=CONFIG.forgetting)
report = mdl.fit(counts=counts, embeds=embeds, time_index=times, max_iter=CONFIG.em_max_iter, tol=CONFIG.em_tol)

mh = MHNamingGame(mdl, alpha=CONFIG.mh_alpha)
mh_log = mh.step(counts=counts, embeds=embeds, time_index=times, trials=CONFIG.mh_trials)
print("MH:", mh_log)


[EM] it=001 NLL=167510.412 JS=0.298
[EM] it=002 NLL=-556767.406 JS=0.378
[EM] it=003 NLL=-617171.301 JS=0.413
[EM] it=004 NLL=-639090.477 JS=0.459
[EM] it=005 NLL=-650894.396 JS=0.471
[EM] it=006 NLL=-654285.966 JS=0.471
[EM] it=007 NLL=-654285.969 JS=0.471
[EM] it=008 NLL=-654285.969 JS=0.471
[EM] Converged.
MH: MHLog(trials=10, accepted=0, accept_rate=0.0, ll_before=654285.968699208, ll_after=654285.968699208)


/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/4011821221.py:94: RuntimeWarning: divide by zero encountered in matmul
  p.lambdas = (gamma.T @ counts) / Nk[:, None]
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/4011821221.py:94: RuntimeWarning: overflow encountered in matmul
  p.lambdas = (gamma.T @ counts) / Nk[:, None]
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/4011821221.py:94: RuntimeWarning: invalid value encountered in matmul
  p.lambdas = (gamma.T @ counts) / Nk[:, None]
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/4011821221.py:98: RuntimeWarning: divide by zero encountered in matmul
  p.mu = (gamma.T @ embeds) / Nk[:, None]
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/4011821221.py:98: RuntimeWarning: overflow encountered in matmul
  p.mu = (gamma.T @ embeds) / Nk[:, None]
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/4011821221.py:98: RuntimeWarning: invalid v


### 6. 因果推定（IPW + 二段推定）

**目的**：横断媒体ラグの**総効果**→**媒介**→**直接**の分解。  
- **治療 \(X_{t-\ell}\)**：例「TikTok の総量（support+critic+neutral）」のラグ  
- **アウトカム \(Y_t\)**：例「X（旧 Twitter）の総量」  
- **媒介 \(M_t\)**：例「X の stance バランス（support−critic）」  
- **交絡**：他媒体のラグ・トレンドなどを**共変量**に、**IPW**で残余バイアスを軽減。

> 実務では `DoubleML` や `DoWhy` などで厳密化し、外生ショックや IV を検討ください。


In [7]:

# =============================================================
# 6. IPW-weighted two-stage causal mediation (from scratch)
# =============================================================
def design_lagged(df: pd.DataFrame, target_source: str, treat_source: str, lag: int = 1) -> pd.DataFrame:
    # Aggregate per time & source
    g = df.groupby(["t","source"], as_index=False)[["support","critic","neutral"]].sum()
    piv = g.pivot(index="t", columns="source", values=["support","critic","neutral"]).fillna(0.0)
    piv.columns = [f"{a}_{b}" for a,b in piv.columns]
    piv = piv.reset_index()

    # treatment: total of treat_source at lag
    piv[f"X_{treat_source}"] = (piv[f"support_{treat_source}"] + piv[f"critic_{treat_source}"] + piv[f"neutral_{treat_source}"]).shift(lag)
    # outcome: total of target_source at t
    piv[f"Y_{target_source}"] = (piv[f"support_{target_source}"] + piv[f"critic_{target_source}"] + piv[f"neutral_{target_source}"])
    # mediator: stance balance of target at t
    piv[f"M_{target_source}"] = (piv[f"support_{target_source}"] - piv[f"critic_{target_source}"])
    piv["trend"] = np.arange(len(piv))

    # controls: other sources totals at lag
    for s in CONFIG.sources:
        if s != treat_source:
            piv[f"Z_{s}"] = (piv[f"support_{s}"] + piv[f"critic_{s}"] + piv[f"neutral_{s}"]).shift(lag)

    piv = piv.dropna().reset_index(drop=True)
    return piv

def add_const(X: np.ndarray) -> np.ndarray:
    return np.hstack([np.ones((X.shape[0],1)), X])

def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1.0/(1.0 + np.exp(-x))

def logistic_fit(X: np.ndarray, y: np.ndarray, sample_weight: Optional[np.ndarray] = None,
                 max_iter: int = 200, tol: float = 1e-8, l2: float = 1e-6) -> np.ndarray:
    '''Newton-Raphson for weighted logistic regression with L2 ridge regularization.'''
    n, p = X.shape
    w = np.zeros(p)
    for _ in range(max_iter):
        z = X @ w
        p_hat = sigmoid(z)
        if sample_weight is None:
            sw = np.ones(n)
        else:
            sw = sample_weight
        # gradient and Hessian
        g = X.T @ (sw * (p_hat - y)) + l2 * w
        W = sw * p_hat * (1 - p_hat)
        H = X.T @ (W[:, None] * X) + l2 * np.eye(p)
        try:
            step = np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            step = np.linalg.lstsq(H, g, rcond=None)[0]
        w_new = w - step
        if np.linalg.norm(w_new - w) < tol:
            w = w_new
            break
        w = w_new
    return w

def wls_fit(X: np.ndarray, y: np.ndarray, weight: Optional[np.ndarray] = None) -> Tuple[np.ndarray, np.ndarray]:
    if weight is None:
        W = np.eye(X.shape[0])
    else:
        W = np.diag(weight)
    XtW = X.T @ W
    beta = np.linalg.lstsq(XtW @ X, XtW @ y, rcond=None)[0]
    # Robust (HC1) covariance
    resid = y - X @ beta
    S = np.zeros((X.shape[1], X.shape[1]))
    for i in range(X.shape[0]):
        xi = X[i:i+1,:]
        S += (weight[i] if weight is not None else 1.0) * (resid[i]**2) * (xi.T @ xi)
    XtWX_inv = np.linalg.inv(XtW @ X + 1e-9*np.eye(X.shape[1]))
    cov = XtWX_inv @ S @ XtWX_inv
    se = np.sqrt(np.diag(cov))
    return beta, se

# Build design for (TikTok -> X) as an example
lag = 1
piv = design_lagged(data.counts, target_source="X", treat_source="TikTok", lag=lag)

# Treatment as binary: high vs low TikTok volume (median split)
X_treat = piv[f"X_TikTok"].to_numpy()
thr = np.median(X_treat)
T_binary = (X_treat >= thr).astype(float)

# Controls
ctrl_cols = ["trend"] + [f"Z_{s}" for s in CONFIG.sources if s != "TikTok"]
X_ctrl = piv[ctrl_cols].to_numpy()

# Propensity score model: Pr(T=1 | controls)
X_p = add_const(X_ctrl)
w_logit = logistic_fit(X_p, T_binary)
ps = sigmoid(X_p @ w_logit)
ps = np.clip(ps, 1e-3, 1-1e-3)

# Stabilized IPW
p_t = np.mean(T_binary)
sw = (p_t * T_binary / ps) + ((1-p_t) * (1-T_binary) / (1-ps))

# Stage 1: mediator M <- T + controls (IPW WLS)
M = piv[f"M_X"].to_numpy()
X1 = add_const(np.column_stack([T_binary, X_ctrl]))
b1, se1 = wls_fit(X1, M, weight=sw)

# Stage 2: outcome Y <- T + M + controls (IPW WLS)
Y = piv[f"Y_X"].to_numpy()
X2 = add_const(np.column_stack([T_binary, M, X_ctrl]))
b2, se2 = wls_fit(X2, Y, weight=sw)

beta_total = wls_fit(add_const(np.column_stack([T_binary, X_ctrl])), Y, weight=sw)[0][1]
delta = b1[1]         # effect of T on M
theta = b2[2]         # effect of M on Y (controlling T, controls)
beta_direct = b2[1]   # direct effect of T on Y
beta_indirect = delta * theta

print("=== Causal Effects (IPW two-stage) ===")
print(f"Total effect (approx): {beta_total:.3f}")
print(f"Direct effect        : {beta_direct:.3f}  (se~{se2[1]:.3f})")
print(f"Indirect (delta*theta): {beta_indirect:.3f}  [delta={delta:.3f}, theta={theta:.3f}]")


=== Causal Effects (IPW two-stage) ===
Total effect (approx): 2.693
Direct effect        : 1.903  (se~1.388)
Indirect (delta*theta): 0.789  [delta=1.990, theta=0.397]


/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/2080998776.py:39: RuntimeWarning: divide by zero encountered in matmul
  z = X @ w
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/2080998776.py:39: RuntimeWarning: overflow encountered in matmul
  z = X @ w
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/2080998776.py:39: RuntimeWarning: invalid value encountered in matmul
  z = X @ w
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/2080998776.py:48: RuntimeWarning: divide by zero encountered in matmul
  H = X.T @ (W[:, None] * X) + l2 * np.eye(p)
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/2080998776.py:48: RuntimeWarning: overflow encountered in matmul
  H = X.T @ (W[:, None] * X) + l2 * np.eye(p)
/var/folders/1q/vxvh30zd7k3brdcpls5z7gq00000gn/T/ipykernel_27313/2080998776.py:48: RuntimeWarning: invalid value encountered in matmul
  H = X.T @ (W[:, None] * X) + l2 * np.eye(p)
/var/folders/1q/vxvh30zd


### 7. Active Inference（MC 近似 EIG）

**候補（媒体×時間窓）**ごとに、モデル \(p(x|w)\) から**サンプル**し、  
**混合事前** \( \pi \) と**事後** \( q(z|x) \) の**エントロピー差**で **情報利得**
\(
\mathrm{EIG} \approx H(\pi) - \mathbb{E}_{x\sim p(x)}[H(q(z|x))]
\)
を近似。上位の窓を**次取得候補**に提示（コストは未考慮）。


In [8]:

# =============================================================
# 7. Active Inference: Monte Carlo EIG per (source, time) candidate
# =============================================================
def entropy(p: np.ndarray) -> float:
    p = np.clip(p, 1e-12, 1.0)
    return float(-np.sum(p * np.log(p)))

def sample_from_model(params: CPCParams, size: int = 1) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    '''Sample (z cluster index, counts C, embeds D) from the current model.'''
    K, C, D = params.lambdas.shape[0], params.lambdas.shape[1], params.mu.shape[1]
    # mixture component choice
    z = np.random.choice(K, size=size, p=params.pi)
    # counts ~ Pois(lam[z])
    counts = np.zeros((size, C))
    embeds = np.zeros((size, D))
    for i in range(size):
        k = z[i]
        counts[i] = np.random.poisson(params.lambdas[k])
        embeds[i] = np.random.normal(loc=params.mu[k], scale=np.sqrt(params.sigma2[k]), size=(D,))
    return z, counts, embeds

def responsibilities(model: CPCMixture, counts: np.ndarray, embeds: np.ndarray) -> np.ndarray:
    p = model.params
    log_like = (
        model._log_poisson(counts, p.lambdas) +
        model._log_gauss(embeds, p.mu, p.sigma2) +
        np.log(p.pi[None, :] + 1e-12)
    )
    # logsumexp
    m = np.max(log_like, axis=1, keepdims=True)
    gamma = np.exp(log_like - (m + np.log(np.sum(np.exp(log_like - m), axis=1, keepdims=True))))
    return gamma  # (N,K)

def eig_for_candidate(model: CPCMixture, mc: int = 64) -> float:
    prior_H = entropy(model.params.pi)
    zs, cts, embs = sample_from_model(model.params, size=mc)
    gamma = responsibilities(model, cts, embs)  # (mc,K)
    post_H = np.mean([entropy(g) for g in gamma])
    return float(prior_H - post_H)

eig_est = eig_for_candidate(mdl, mc=CONFIG.active_mc_samples)
print(f"[Active Inference] Approx. EIG per sample ~ {eig_est:.4f} nats")

# 実運用では (source, t) ごとに、近傍データの統計から λ を微調整したローカル予測分布で EIG を計算します。


[Active Inference] Approx. EIG per sample ~ 1.2686 nats



### 8. ホールドアウト評価と可視化

- 時系列終盤を **Hold‑out** にして**予測対数尤度**を評価。  
- 簡易可視化：クラスタ中心、割当、NLL 推移など。


In [9]:

# =============================================================
# 8. Hold-out evaluation (predictive log-likelihood)
# =============================================================
def train_test_split_by_time(df: pd.DataFrame, holdout_ratio: float = 0.2) -> Tuple[np.ndarray, np.ndarray]:
    ts = df["t"].to_numpy()
    t_thr = np.quantile(ts, 1.0 - holdout_ratio)
    train_idx = np.where(ts <= t_thr)[0]
    test_idx = np.where(ts > t_thr)[0]
    return train_idx, test_idx

train_idx, test_idx = train_test_split_by_time(df, holdout_ratio=CONFIG.holdout_ratio)

ll_train = mdl.log_likelihood(counts[train_idx], embeds[train_idx])
ll_test  = mdl.log_likelihood(counts[test_idx],  embeds[test_idx])

print(f"Train log-likelihood: {ll_train:.2f}  |  Test log-likelihood: {ll_test:.2f}")

# （必要に応じて実行）プロット例：中心間距離のヒストグラム
def plot_center_norms(mu: np.ndarray):
    norms = np.linalg.norm(mu, axis=1)
    plt.figure()
    plt.hist(norms, bins=10)
    plt.title("Cluster center norms")
    plt.xlabel("||w_j||")
    plt.ylabel("freq")
    plt.show()

# plot_center_norms(mdl.params.mu)  # 実行は任意


Train log-likelihood: 522561.76  |  Test log-likelihood: 131724.21



### 9. チェックポイントと安全化シリアライズ

- **モデル**：\(\pi, \lambda, \mu, \sigma^2\) を **NPZ** で保存/読込  
- **メタ情報**：JSON（Config、指標、日付）


In [10]:

# =============================================================
# 9. Checkpointing
# =============================================================
from datetime import datetime

def save_checkpoint(model: CPCMixture, path_prefix: str, meta: Dict):
    p = model.params
    np.savez_compressed(
        path_prefix + ".npz",
        pi=p.pi, lambdas=p.lambdas, mu=p.mu, sigma2=p.sigma2
    )
    with open(path_prefix + ".json", "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

def load_checkpoint(path_prefix: str) -> Tuple[CPCParams, Dict]:
    z = np.load(path_prefix + ".npz")
    with open(path_prefix + ".json", "r", encoding="utf-8") as f:
        meta = json.load(f)
    return CPCParams(pi=z["pi"], lambdas=z["lambdas"], mu=z["mu"], sigma2=z["sigma2"]), meta

ckpt_prefix = os.path.join(CONFIG.out_dir, "cpc_mixture_ckpt")
os.makedirs(CONFIG.out_dir, exist_ok=True)

# Save checkpoint after one full training pass + MH step
save_checkpoint(mdl, ckpt_prefix, meta={
    "created_at": datetime.utcnow().isoformat(),
    "config": {k: (list(v) if isinstance(v, tuple) else v) for k, v in CONFIG.__dict__.items()},
    "em_report": {k: (float(v) if isinstance(v, (np.floating, np.float32, np.float64)) else v) for k,v in report.__dict__.items()},
    "mh": {k: (float(v) if isinstance(v, (np.floating, np.float32, np.float64)) else v) for k,v in mh_log.__dict__.items()}
})
print("Checkpoint saved:", ckpt_prefix + ".npz / .json")


Checkpoint saved: artifacts/cpc_mixture_ckpt.npz / .json



### 付録：API 拡張ポイント

- **埋め込み差し替え**：`HashedTfidf` を、`SentenceTransformer` 等に置換（`fit/transform` 互換）。  
- **尤度の強化**：`CPCMixture` を **Negative Binomial**（過分散）や**混合ガウス（完全対角/フル）**に拡張。  
- **MH 提案分布**：`_cluster_semantic_direction` を **LLM 名称埋め込み**に置換、**split/merge** 提案を追加。  
- **因果**：`logistic_fit, wls_fit` を `sklearn` / `statsmodels` に置換し、**HAC** や **IV** を導入。  
- **Active Inference**：EIG に **コスト** \(G+\lambda\,\mathrm{cost}\) を組込み、**ベイズ最適化**で探索。

---

**ライセンス**：このノートブックは教育・実験目的で提供されます。責任を持ってご利用ください。
